In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rnd = np.random
rnd.seed(0)

In [ ]:
p = 2 # nombre d'H
m = 3 # nombre de m'ambulance (VABSAN)
n = 6 # nombre de P

H = [i for i in range(0,p)] # liste des hopitaux
K = [i for i in range(p,p+m)] # liste des ambulances
P = [i for i in range(p+m,p+m+n)] # liste des patients
N = H + K + P

Q = 4 # capacité homogène des ambulances
q = {i:1 for i in P}
ch = [4, 4] # capacité des hopitaux

In [ ]:
# proportion P [alpha, bravo, charlie]
proportion = [0.2, 0.4, 0.4] # proportion des code de P
liste = ["R", "Y", "G"] # code des P Red, Yellow and Green
dist = np.random.choice(liste, n, proportion) # repartition des codes P

codeP = {P[i]:dist[i] for i in range(0,n)} # liste des P avec leur code initial
xc_i = {} # état du patient i (0 assis, 1 couché)
l_i = {} # temps limite d'évacuation du patient i vers un hôpital
s_i = {} # temps de prise en charge du patient i

j=0
for i in dist :
    if i == "R":
        xc_i.update({P[j]:1})
        l_i.update({P[j]:90})
        s_i.update({P[j]:10})
        j+=1
    elif i == "Y":
        xc_i.update({P[j]:1})
        l_i.update({P[j]:240})
        s_i.update({P[j]:10})
        j+=1
    else:
        xc_i.update({P[j]:rnd.randint(0,2)})
        l_i.update({P[j]:1440})
        s_i.update({P[j]:5})
        j+=1


In [ ]:
loc_x = rnd.rand(len(N))*20
loc_y = rnd.rand(len(N))*10
# loc_x[p]=loc_x[0]
# loc_x[p+1]=loc_x[0]
# loc_x[p+2]=loc_x[1]
# loc_y[p]=loc_y[0]
# loc_y[p+1]=loc_y[0]
# loc_y[p+2]=loc_y[1]

In [ ]:
plt.scatter(loc_x[0:p], loc_y[0:p], c='r', marker='s')
for i in H:
    plt.annotate('$h_%d=%d$' % (i+1, ch[i]), (loc_x[i], loc_y[i]-5))
    
for i in K:
    plt.plot(loc_x[i], loc_y[i], c='b', marker='x')
    plt.annotate('$a_{0!s} = {1!s}$'.format(i-(p-1), Q), (loc_x[i], loc_y[i]+i*i))


for i in P:
    plt.plot(loc_x[i], loc_y[i], c='k', marker='o')
    plt.annotate('p_{0!s}={1!s}'.format(i-p-1, xc_i[i]), (loc_x[i]-3, loc_y[i]-5))
plt.axis('equal')

In [ ]:
capacite ={}
for h in H:
    capacite.update({h:6})
for k in K:
    capacite.update({k:4})

In [ ]:
KxP = [(i,j) for i in K for j in P]
PxP = [(i,j) for i in P for j in P]
PxH = [(i,j) for i in P for j in H]

In [ ]:
print(KxP)

In [ ]:
A = KxP+PxP+PxH
tij={(i,j):np.hypot(loc_x[i]-loc_x[j], loc_y[i]-loc_y[j])*5 for i in N for j in N}

In [ ]:
print(A)
print(len(tij))

In [ ]:
from docplex.mp.model import Model

In [ ]:
m = Model('VABSAN')

In [ ]:
PUK = P + K
PUH = H + P
HUK = K + H

In [ ]:
print(PUK)
print([i for i in PUK])

In [ ]:
x = m.binary_var_cube(PUK, N, K, name='x')
d_k = m.continuous_var_dict(K, lb=0, ub=1440,  name = 'temps_service')
y = m.binary_var_cube(P,H,K, name = 'patient_ambulance_hopital')
u = m.integer_var_dict(P, ub=Q, name='u')

In [ ]:
#----- Objective Function -----
#Minimize the total cost
function1 = m.sum(x[i,j,k] for j in P for i in PUK for k in K)
function2 = m.sum(tij[(i,j)]*x[i,j,k] for j in N for i in PUK for k in K)
function3 = m.sum(x[i,j,k]/(l_i[j]-s_i[j])  for j in P for i in PUK for k in K)


sense="min"
exprs=[-function1,function2,-function3]
priorities=[1,3,2]
weights=[1,1,1]
m.set_multi_objective(sense, exprs, priorities, weights, abstols=None, reltols=None, names=None)


In [ ]:
m.cts_1 = []
for k in K: 
    ct_1 = m.sum(
        x[i,j,k] for i in K if i != k for j in P)== 0
    ct_1.name = 'contrainte_ambulance_{0!s}'.format(k)
    m.cts_1.append(ct_1)

m.add_constraints(m.cts_1)

# print('cts_1=', m.cts_1)

In [ ]:
m.cts_2 = []
for i in P : 
    ct_2= m.sum(
        x[i,j,k] for j in PUH for k in K) <= 1
    ct_2.name = 'contrainte_patient_{0!s}'.format(i)
    m.cts_2.append(ct_2)

m.add_constraints(m.cts_2)

# print('cts_2=', m.cts_2)

In [ ]:
m.cts_3 = []
for j in P : 
    ct_3= m.sum(
        x[i,j,k] for i in PUK for k in K) <= 1
    ct_3.name = 'contrainte_patient_2_{0!s}'.format(j)
    m.cts_3.append(ct_3)

m.add_constraints(m.cts_3)

# print('cts_3=', m.cts_3)

In [ ]:
m.cts_4 = []
for j in P:
    for k in K:
        ct_4 = (m.sum(x[i,j,k] for i in PUK) - 
            m.sum(x[j,i,k] for i in PUH) == 0)
        ct_4.name = 'flow_constraint_{0!s}_{1!s}'.format(j,k)
        m.cts_4.append(ct_4)

m.add_constraints(m.cts_4)

# print('cts_4=', m.cts_4)

In [ ]:
m.cts_6 = []
for k in K:
    for z in P:
        ct_6 = ((d_k[k] + m.sum(tij[(i,j)]*x[i,j,k] for i in PUK for j in N if i!=j) +
                m.sum(x[i,j,k]*s_i[j] for i in PUK for j in P)) <= l_i[z])
        ct_6.name = 'constraint_temps_{0!s}_{1!s}'.format(k,z)
        m.cts_6.append(ct_6)

m.add_constraints(m.cts_6)


In [ ]:
m.cts_7 = []
for k in K : 
    ct_7= m.sum(
        x[i,j,k] for i in PUK for j in P) <= capacite[k]
    ct_7.name = 'contrainte_capacite_ambulance{0!s}'.format(k)
    m.cts_7.append(ct_7)

m.add_constraints(m.cts_7)

m.add_indicator_constraints(m.indicator_constraint(x[i,j,k], u[i]+q[i]==u[j]) 
                                            for i, j in A if i in P and j in P for k in K)

m.add_constraints(u[i]>=q[i]+xc_i[i] for i in P)

# print('cts_7=', m.cts_7)

In [ ]:
m.cts_8 = []
for k in K : 
    for i in K :
        ct_8= m.sum(x[i,j,k] for j in P) <= 1
        ct_8.name = 'contrainte_prise_en_charge_{0!s}_{1!s}'.format(k,i)
        m.cts_8.append(ct_8)

m.add_constraints(m.cts_8)

print('cts_8=', m.cts_8)

In [ ]:
# for i in P:
#     for h in H : 
#         for k in K:
#             # ct_9 = 
#             y[i,h,k] = m.min(
#                 m.sum(x[i,j,k] for j in PUH),
#                 m.sum(x[j,h,k] for j in P))
m.add_indicator_constraints(m.indicator_constraint(x[i,j,k], y[i,h,k] == m.min(
                m.sum(x[i,j,k] for j in PUH),
                m.sum(x[j,h,k] for j in P))) 
                                            for i in P for h in H for k in K)

       

In [ ]:
m.cts_10 = []
for h in H : 
    ct_10 = m.sum(y[i,h,k] for k in K for i in P) <= capacite[h]
    ct_10.name = 'contrainte_capacite_H{0!s}'.format(h)
    m.cts_10.append(ct_10)

m.add_constraints(m.cts_10)

In [ ]:
m.cts_11 = []
for i in P: 
    ct_11 = m.sum(
        x[i,j,k] for j in P if i == j )== 0
    ct_11.name = 'contrainte_patient_patient_{0!s}'.format(k)
    m.cts_11.append(ct_11)

m.add_constraints(m.cts_11)

# print('cts_1=', m.cts_1)

In [ ]:
sol = m.solve(log_output = True)
print(sol)

In [ ]:
active_arcs = []
for k in K:
    for i,j in A:
        if x[i,j,k].solution_value >0.9:
            active_arcs.append([x[i,j,k]])


In [ ]:
active_arcs

In [ ]:
plt.scatter(loc_x[0:p], loc_y[0:p], c='r', marker='s')
for i in H:
    plt.annotate('$h_%d=%d$' % (i+1, ch[i]), (loc_x[i], loc_y[i]-5))
    
for i in K:
    plt.plot(loc_x[i], loc_y[i], c='b', marker='x')
    plt.annotate('$a_{0!s} = {1!s}$'.format(i-(p-1), Q), (loc_x[i], loc_y[i]+i*i))


for i in P:
    plt.plot(loc_x[i], loc_y[i], c='k', marker='o')
    plt.annotate('p_{0!s}={1!s}'.format(i-p-1, xc_i[i]), (loc_x[i]-3, loc_y[i]-5))
plt.axis('equal')